In [7]:
from copy import deepcopy
import math
from random import randint
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import tkinter

In [8]:
class Etat : # c'est la classe qui décrit nos différents états pour la recherche informée
    def __init__(self, visited=[], tovisit=[], startingcity = '', currentcity='', costs=[], cost=0, value = (-1), cities=[], hist = []):
        self.visited = visited #les villes deja visitees on pourrait utiliser une liste
        self.tovisit = tovisit # les villes a visiter, une liste pareil
        self.startingcity = startingcity # ville de départ qui devre etre la ville d'arrivee
        self.currentcity = currentcity # la ville ou on en est, on pourrait aussi faire self.visited.tail peut être
        self.costs = costs # une liste de liste [[a,b,3], [a,c,5], etc...]
        self.cost = cost # le coût de ce chemin à l'étape currentcity servira de g(n)
        self.value = value # resultat de f = h + g
        self.cities = cities
        self.hist = hist # etat precedent
        
    def addpath(self, path): #path de la forme ['Paris','Lyon',45] et l'ajoute à self
        nouv = deepcopy(self)
        nouv.hist.append([path[0], path[1]]) # on met à jour l'historique de chaque etat
        if (path[0] in nouv.visited):
            nouv.currentcity = path[1]
        else:
            nouv.currentcity = path[0]
        nouv.visited.append(path[0])
        nouv.visited.append(path[1])
        nouv.visited = list(set(nouv.visited))
        nouv.cost+= path[2]
        if (path[0] in nouv.tovisit ) : nouv.tovisit.remove(path[0]) # ce qui est visité n'est plus à visiter
        if (path[1] in nouv.tovisit ) :nouv.tovisit.remove(path[1])
        return nouv 

    def getmintree(self):
        cop = self.costs.copy()
        vis = self.visited
        edg = []
        cos = self.cost # totalcost
        index = 0
        if len(vis)==0: # to get started
            for i in range(len(cop)):
                for j in range(len(cop)):
                    if (cop[i][2]<=cop[j][2]) and (cop[index][2]>cop[i][2]):
                        index = i    
            cos+=cop[index][2] # le cout total
            edg.append(cop[index])
            vis.extend((cop[index][0],cop[index][1])) # on met les villes visited dans vis
        if len(vis) != 0:
            for k in range(len(self.cities)-len(vis)):
                index = 0
                for i in range(len(cop)):
                    if not((cop[i][0] in vis) and (cop[i][1] in vis)) and not((cop[i][0] not in vis) and (cop[i][1] not in vis)):
                        for j in range(len(cop)):
                            if not((cop[j][0] in vis) and (cop[j][1] in vis)) and not((cop[j][0] not in vis) and (cop[j][1] not in vis)): # ici on a l'ensemble frontière
                                if (cop[i][2]<=cop[j][2]) and (cop[index][2]>cop[i][2]):
                                    index = i
                vis.extend((cop[index][0],cop[index][1]))
                vis = list(set(vis)) #pour supprimer les doublons
                cos+=cop[index][2]
                edg.append(cop[index])
                self.currentcity = edg[-1][1]
                self.visited = vis
                self.cost = cos
                self.tovisit = list(set(self.tovisit)- set(self.visited))
        return edg

    def heuristic(self): # sera notre heuristic via calcul du minspantree
        nouv = deepcopy(self)
        cop = nouv.getcosts()
        vis = nouv.getvisited()
        edg = []
        cos = nouv.getcost() # totalcost
        index = 0
        frontier = []
        if len(vis)==0: # to get started
            for i in range(len(cop)):
                for j in range(len(cop)):
                    if (cop[i][2]<=cop[j][2]) and (cop[index][2]>cop[i][2]):
                        index = i    
            cos+=cop[index][2] # le cout total
            edg.append(cop[index])
            vis.extend((cop[index][0],cop[index][1])) # on met les villes visited dans vis
        if len(vis) != 0:
            for k in range(len(nouv.getcities())-len(vis)):
                index = 0
                for i in range(len(cop)):
                    if not((cop[i][0] in vis) and (cop[i][1] in vis)) and not((cop[i][0] not in vis) and (cop[i][1] not in vis)):
                        if (k == 0): #la frontier pour le premier coup
                            frontier.append(cop[i]) # les edges encore candidats
                        for j in range(len(cop)):
                            if not((cop[j][0] in vis) and (cop[j][1] in vis)) and not((cop[j][0] not in vis) and (cop[j][1] not in vis)): # ici on a l'ensemble frontière
                                if (cop[i][2]<=cop[j][2]) and (cop[index][2]>cop[i][2]):
                                    index = i
                vis.extend((cop[index][0],cop[index][1]))
                vis = list(set(vis)) #pour supprimer les doublons
                cos+=cop[index][2]
                edg.append(cop[index])
                nouv.currentcity = edg[-1][1]
                nouv.visited = vis
                nouv.tovisit = list(set(nouv.tovisit)- set(nouv.visited))
                new_f = []
                for elem in frontier:
                    if elem not in new_f:
                        new_f.append(elem)
                frontier = new_f # il y a toutes les frontières ici mais on peut voir à les avoir pour une certaine étape ou faire une liste de liste
        return (cos - nouv.getcost())

    def frontier(self): # va retourner les etats frontier qui peuvent etre ajouté à l'arbre mais pas forcément dans astar comme on doit partie de la dernière ville visitée
        front = []
        vis = self.visited.copy()
        cop = self.costs.copy()
        for i in range(len(cop)):
            if not((cop[i][0] in vis) and (cop[i][1] in vis)) and not((cop[i][0] not in vis) and (cop[i][1] not in vis)): #la frontier pour le premier coup
                front.append(cop[i]) # les edges encore candidats
        new_f = []
        for elem in front:
            if elem not in new_f:
                new_f.append(elem)
        front = new_f
        return front

    def openfront(self):
        op = []
        vis = self.getvisited()
        cop = self.costs.copy()
        for i in range(len(cop)):
            if ((cop[i][0] == self.getcurrentcity()) and (cop[i][1] not in vis)) or ((cop[i][1] == self.getcurrentcity()) and (cop[i][0] not in vis)): #on veut que les coups suivants partent de la dernière ville visitée
                op.append(cop[i]) # les edges encore candidats
        return op


    def addvisited(self, city): # serves to add a city to the visited ones
        try:
            assert(city in self.tovisit)
            self.visited.append(city)
        except:
            print("Cette ville n'est pas dans la liste des villes encore disponibles \n Essayer la fonction ShowUnvisitedCities")

    def getvisited(self):
        return self.visited

    def getcosts(self):
        return self.costs

    def getcost(self):
        return self.cost

    def getcurrentcity(self):
        return self.currentcity

    def getstartingcity(self):
        return self.startingcity

    def getcities(self):
        return self.cities
    
    def gethist(self):
        return self.hist

    def show_unvisited_cities(self):
        print(f"There are the cities left to visit {self.tovisit}")

    def prettyprint(self):
        print('Villes déjà visitées :' + str(self.visited))
        print('Villes à visiter :' + str(self.tovisit))
        print('Ville de départ :' + self.startingcity)
        print('Villeactuelle :' + self.currentcity)
        print('Données :' + str(self.costs))
        print('Coût à cette étape :' + str(self.cost))
        print('valeur de f = h + g :' + str(self.value))
        print('Toutes les villes :' + str(self.cities))
        print('Historique des chemins empreintés :' + str(self.hist))
        
    def setvalue(self):
        self.value = self.cost + self.heuristic()
    
    def getvalue(self):
        return self.value

Ci dessous nous avons la fonction astar qui met applique l'algorithme eponyme. Il peut gérer jusqu'à un graph complet de 10 villes dans un temps raisonnable. Pour des graphs plus grand le temps d'execution devient trop long.

In [9]:
def astar(etat):
    cop = deepcopy(etat)
    cop.setvalue()
    op = [cop] #la frontière, à voire comment représenter ça, ce sera une liste d'états, à ranger dans l'ordre croissant de f = g + h ou alors avoir une fonction qui nous donne le min
    maxop = op.copy()
    closed = [] # c'est les états validés et ce sera le resultat final je pense
    closed.append(op.pop()) # initialisation
    toadd = closed[-1].openfront() # on récupère les étapes suivantes possibles
    for i in toadd:
        temp = deepcopy(closed[-1]) # et on crée les etats correspondants
        tempi = temp.addpath(i)
        tempi.setvalue()
        op.append(tempi) # on continue de peupler op
    while (len(closed[-1].getvisited()) != len(etat.getcities())): # tant qu'on a pas visité toutes les villes
        indice = 0
        indsuppr = 0
        first = math.inf
        for i in op: # on va prendre dans indice l'indice de ce qui nous intéresse dans op sans oublier de le suppr de op
            indice+=1
            val = i.getvalue()
            if (val)< first:
                first = val
                indsuppr = indice
        closed.append(op.pop(indsuppr-1)) #on déplace ce qui nous interesse de op vers closed sans oublier de le suppr de op
        toadd = closed[-1].openfront() # on récupère les étapes suivantes possibles
        for i in toadd:
            temp = deepcopy(closed[-1]) # et on crée les etats correspondants
            tempi = temp.addpath(i)
            tempi.setvalue()
            op.append(tempi) # on continue de peupler op
            if len(op) > len(maxop):
                maxop = op
    chemin = closed[-1].gethist()
    print('La taille max de l\'ensemble frontière est de', len(maxop))
    last = chemin[-1][0] # dernière ville visitée
    if chemin[-1][0] in chemin[-2]:
        last = chemin[-1][1]
    recherche = closed[-1].getcosts() # le graph
    derniercout = 0
    for i in recherche:
        if (i[0] == closed[-1].getstartingcity()) and (i[1] == last):
            derniercout = i[2]
    der = closed[-1].addpath([closed[-1].getstartingcity(),last, derniercout])
    print('Le cout finalement trouvé est de ', der.getcost())
    chemin = der.gethist()
    print(chemin)
    return chemin
    

Nous allons maintenant appliquer l'algorithme astar pour un graph généré aléatoirement. vous pouvez choisir le nombre de villes qu'il y a dans le graph. C'est rapide jusqu'à 8 puis je n'ai pas mis plus de 11 car ça devient trop long.

In [10]:
villeFrance = ['Paris','Marseille','Lyon','Nantes','Rennes','Toulouse','Bordeaux','Lille','Fontainebleau','Saint-Malo','Verdun','Nice','Strasbourg',
'Montpellier','Aix-en-Provence','Angers','Dijon','Grenoble','Annecy','Avignon','Nîmes','Le Havre','Toulon','Brest','Tours',
'Clermont-Ferrand','Rouen','Limoges','Caen','Le Mans','Nancy','Amiens','Metz','Perpignan','Orléans','Besancon','Vichy','Mulhouse','Tourcoing',
'Poitiers','Saint-Denis','Roubaix','Montreuil','Sevran','Nanterre','Cannes','La Rochelle', 'Biarritz','Cabourg','Deauville']

In [11]:
def make(nb):
    villeFrance = ['Paris','Marseille','Lyon','Nantes','Rennes','Toulouse','Bordeaux','Lille','Fontainebleau','Saint-Malo','Verdun','Nice','Strasbourg',
                    'Montpellier','Aix-en-Provence','Angers','Dijon','Grenoble','Annecy','Avignon','Nîmes','Le Havre','Toulon','Brest','Tours',
                    'Clermont-Ferrand','Rouen','Limoges','Caen','Le Mans','Nancy','Amiens','Metz','Perpignan','Orléans','Besancon','Vichy','Mulhouse','Tourcoing',
                    'Poitiers','Saint-Denis','Roubaix','Montreuil','Sevran','Nanterre','Cannes','La Rochelle', 'Biarritz','Cabourg','Deauville']
    graph = []
    avisiter = []
    x=np.random.randint(50, size=(nb))
    depart = villeFrance[0]
    for i in range(nb):
        avisiter.append(villeFrance[i])
        for j in range (i+1, nb):
            graph.append([villeFrance[i],villeFrance[j], randint(1, 100)])#On dit arbitrairement que la distance est comprise entre 1 et 100
    ori = Etat([depart],avisiter, depart,depart, graph, 0, 0, avisiter, [])
    return astar(ori)


interact(make, nb=widgets.Dropdown(options=range(3,12),description = 'Nb de villes', value=5))

interactive(children=(Dropdown(description='Nb de villes', index=2, options=(3, 4, 5, 6, 7, 8, 9, 10, 11), val…

<function __main__.make(nb)>

In [13]:
def taux(nb):
    villeFrance = ['Paris','Marseille','Lyon','Nantes','Rennes','Toulouse','Bordeaux','Lille','Fontainebleau','Saint-Malo','Verdun','Nice','Strasbourg',
                    'Montpellier','Aix-en-Provence','Angers','Dijon','Grenoble','Annecy','Avignon','Nîmes','Le Havre','Toulon','Brest','Tours',
                    'Clermont-Ferrand','Rouen','Limoges','Caen','Le Mans','Nancy','Amiens','Metz','Perpignan','Orléans','Besancon','Vichy','Mulhouse','Tourcoing',
                    'Poitiers','Saint-Denis','Roubaix','Montreuil','Sevran','Nanterre','Cannes','La Rochelle', 'Biarritz','Cabourg','Deauville']
    graph = []
    avisiter = []
    x=np.random.randint(50, size=(nb))
    depart = villeFrance[0]
    for i in range(nb):
        avisiter.append(villeFrance[i])
        for j in range (i+1, nb):
            graph.append([villeFrance[i],villeFrance[j], randint(1, 100)])#On dit arbitrairement que la distance est comprise entre 1 et 100
    ori = Etat([depart],avisiter, depart,depart, graph, 0, 0, avisiter, [])
    return ori

def astar2(etat):
    cop = deepcopy(etat)
    cop.setvalue()
    op = [cop] #la frontière, à voire comment représenter ça, ce sera une liste d'états, à ranger dans l'ordre croissant de f = g + h ou alors avoir une fonction qui nous donne le min
    maxop = op.copy()
    closed = [] # c'est les états validés et ce sera le resultat final je pense
    closed.append(op.pop()) # initialisation
    toadd = closed[-1].openfront() # on récupère les étapes suivantes possibles
    for i in toadd:
        temp = deepcopy(closed[-1]) # et on crée les etats correspondants
        tempi = temp.addpath(i)
        tempi.setvalue()
        op.append(tempi) # on continue de peupler op
    while (len(closed[-1].getvisited()) != len(etat.getcities())): # tant qu'on a pas visité toutes les villes
        indice = 0
        indsuppr = 0
        first = math.inf
        for i in op: # on va prendre dans indice l'indice de ce qui nous intéresse dans op sans oublier de le suppr de op
            indice+=1
            val = i.getvalue()
            if (val)< first:
                first = val
                indsuppr = indice
        closed.append(op.pop(indsuppr-1)) #on déplace ce qui nous interesse de op vers closed sans oublier de le suppr de op
        toadd = closed[-1].openfront() # on récupère les étapes suivantes possibles
        for i in toadd:
            temp = deepcopy(closed[-1]) # et on crée les etats correspondants
            tempi = temp.addpath(i)
            tempi.setvalue()
            op.append(tempi) # on continue de peupler op
            if len(op) > len(maxop):
                maxop = op
    chemin = closed[-1].gethist()
    last = chemin[-1][0] # dernière ville visitée
    if chemin[-1][0] in chemin[-2]:
        last = chemin[-1][1]
    recherche = closed[-1].getcosts() # le graph
    derniercout = 0
    for i in recherche:
        if (i[0] == closed[-1].getstartingcity()) and (i[1] == last):
            derniercout = i[2]
    der = closed[-1].addpath([closed[-1].getstartingcity(),last, derniercout])
    return der
'''
# on va utiliser addpath(self,path)
# on va regarder pour 7 villes
z = 15 # nb d'itérations
y = 9 # nombre de villes
tot = 0
for k in range(z):
    fin = taux(y)
    fin2 = astar2(fin)  #etat après astar
    c = fin.getcosts()
    nbvilles = len(fin.getcities())
    i=1
    j=0
    while j<len(c):
        fin = fin.addpath(c[j])
        j = j+(nbvilles-i)
        i+=1
    tot = tot + round(((fin2.getcost() - fin.getcost())/fin.getcost())*100, 2)
print('Taux d\'amelioration moyen pour',y,'villes avec astar:', (tot/z))
'''

"\n# on va utiliser addpath(self,path)\n# on va regarder pour 7 villes\nz = 15 # nb d'itérations\ny = 9 # nombre de villes\ntot = 0\nfor k in range(z):\n    fin = taux(y)\n    fin2 = astar2(fin)  #etat après astar\n    c = fin.getcosts()\n    nbvilles = len(fin.getcities())\n    i=1\n    j=0\n    while j<len(c):\n        fin = fin.addpath(c[j])\n        j = j+(nbvilles-i)\n        i+=1\n    tot = tot + round(((fin2.getcost() - fin.getcost())/fin.getcost())*100, 2)\nprint('Taux d'amelioration moyen pour',y,'villes avec astar:', (tot/z))\n"

In [14]:
x=np.random.randint(50, size=(5))
x

array([ 9,  6, 12, 11, 34])

Vous pouvez tester dans la cellule ci dessous avec le nombre de villes que vous voulez mais ce sera trop long sauf coup de chance.

In [17]:
make(5)

La taille max de l'ensemble frontière est de 17
Le cout finalement trouvé est de  187
[['Paris', 'Nantes'], ['Nantes', 'Rennes'], ['Lyon', 'Rennes'], ['Marseille', 'Lyon'], ['Paris', 'Marseille']]


[['Paris', 'Nantes'],
 ['Nantes', 'Rennes'],
 ['Lyon', 'Rennes'],
 ['Marseille', 'Lyon'],
 ['Paris', 'Marseille']]

Passons maintenant à la recherche locale. Ici les villes seront représentées par des numéros et le graph par une matrice. 

In [18]:
#On va prendre pour solution initiale le chemin 0-1-2-3-4-5-

class Local:

    def __init__(self, circuit=[], cost = 0, costs=[]):
        self.circuit = circuit #le chemin qu'on prend
        self.cost = cost #le cout courant
        self.costs = costs # la matrice avec tous les couts

    def getcircuit(self):
        return self.circuit

    def getcost(self):
        return self.cost

    def setcost(self): 
        newcost = 0
        mat = self.getcosts()
        cir = self.getcircuit()
        taille = len(self.getcircuit())
        for i in range(taille-1):
            newcost+= mat[cir[i]][cir[i+1]]
        self.cost = newcost
        
    def getcosts(self):
        return self.costs

    def hillclimb(self):
        res = deuxopt(self.getcircuit(), self.getcosts())
        self.circuit = res
        self.setcost()
        return self.circuit

def deuxopt(route, matgraph):
    result = route
    continuer = True
    taille = len(route)
    while continuer:
        continuer = False
        for i in range(1, taille - 2): #On test tous les voisins dans ces deux boucles on démarre à un car on ne change pas le point de départ
            for j in range(i + 1, taille):
                if (j != (i+1)):# on ignore le cas où i et j sont consécutifs
                    if (matgraph[result[i - 1]][result[i]] + matgraph[result[j - 1]][result[j]]) - (matgraph[result[i - 1]][result[j - 1]] + matgraph[result[i]][result[j]]) > 0: #si on a un cout testé plus intéressant, ce sera notre condition d'arret
                        result[i:j] = result[j - 1:i - 1:-1] #on doit modifier le sens de parcours entre les deux arcs qu'on a modifié
                        continuer = True # si c'est à False c'est qu'on a rien pu modifier pour améliorer l'algo, on est donc dans un creux de notre hill, un minimum local quoi 
    return result

In [19]:
def AlgoHill(loc):
    loc.setcost()
    chInit = deepcopy(loc).getcircuit()
    v1 = loc.getcost()
    loc.hillclimb()
    chFinal = loc.getcircuit()
    v2 = loc.getcost()
    var = round(((v2-v1)/v1)*100,2) # on arrondit le pourcentage pour la lisibilité
    print('La variation a été de', var, '%', 'Avec un coût initial de', v1, 'et un coût finale de', v2)
    print('le chemin initial était :', chInit)
    print('Le chemin final est :', chFinal)


def aff(nb):
    first = list(range(nb))
    matgraph = np.random.randint(100, size=(nb, nb))
    matgraph[range(len(matgraph)), range(len(matgraph))] = 0 # on met des 0 sur la diag 
    matgraph = matgraph + matgraph.T # on additionne la matrice et sa transposée pour avoir une matrice symétrique
    test = Local(first,0, matgraph) # on crée notre instance
    AlgoHill(test)

interact(aff, nb=widgets.Dropdown(options=range(3,1001),description = 'Nb de villes', value=200))

interactive(children=(Dropdown(description='Nb de villes', index=197, options=(3, 4, 5, 6, 7, 8, 9, 10, 11, 12…

<function __main__.aff(nb)>

Pour ce qui est de calculer l'écart à l'optimum il est difficile de le calculer pour un nombre de ville éxcédent ce que l'algorithme A* peut faire de plus la représentation du problème est différente.

In [20]:
def AlgoHill2(loc):
    loc.setcost()
    chInit = deepcopy(loc).getcircuit()
    v1 = loc.getcost()
    loc.hillclimb()
    chFinal = loc.getcircuit()
    v2 = loc.getcost()
    var = round(((v2-v1)/v1)*100,2) # on arrondit le pourcentage pour la lisibilité
    return var


def aff2(nb):
    first = list(range(nb))
    matgraph = np.random.randint(100, size=(nb, nb))
    matgraph[range(len(matgraph)), range(len(matgraph))] = 0 # on met des 0 sur la diag 
    matgraph = matgraph + matgraph.T # on additionne la matrice et sa transposée pour avoir une matrice symétrique
    test = Local(first,0, matgraph) # on crée notre instance
    return(AlgoHill2(test))

# calcul du taux d'amélioration moyen pour le Hillclimb
# on va utiliser addpath(self,path)
# on va regarder pour 7 villes
z = 500 # nb d'itérations
y = 11 # nombre de villes
tot = 0
for k in range(z):
    aff2(y)
    tot = tot + aff2(y)
print('Taux d\'amelioration moyen pour',y,'villes avec HillClimbing:', round(tot/z,3))


Taux d'amelioration moyen pour 11 villes avec HillClimbing: -42.327
